<a href="https://colab.research.google.com/github/NazHub1993/BootExamRock/blob/main/ML_Algorithms(Naive_Bayes_Multinomial).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer


In [114]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [115]:
df['Category']=df['Category'].apply(lambda x:1 if x=='spam' else 0)
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [116]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop('Category',axis='columns'),df.Category)

In [117]:
x_train

,Message
4023,U gd lor go shopping i got stuff to do. U wan ...
1646,I.ll post her out l8r. In class
5422,Ok k..sry i knw 2 siva..tats y i askd..
1254,Mum say we wan to go then go... Then she can s...
3440,"awesome, how do I deal with the gate? Charles ..."
...,...
929,Oh all have to come ah?
1725,There bold 2 &lt;#&gt; . Is that yours
2374,XMAS Prize draws! We are trying to contact U. ...
954,Also remember to get dobby's bowl from your car


##Using the count vectorizer to identify each messsage separately

In [118]:
from sklearn.feature_extraction.text import CountVectorizer
v=CountVectorizer()
x_train_converted=v.fit_transform(x_train.Message)
x_train_converted.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [119]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_converted,y_train)

MultinomialNB()

In [120]:
x_test_converted=v.transform(x_test.Message)
model.score(x_test_converted,y_test)

0.9856424982053122

In [121]:
y_pred=model.predict(x_test_converted)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9856424982053122

In [122]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
email_converted=v.transform(emails)
model.predict(email_converted)

array([0, 1])

#Let's do it using Pipeline

In [123]:
#First you need to declare to which column you should apply using ColumnTransformer only the it would work right

In [124]:
df.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   int64 
 1   Message   5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [126]:
x_train.shape

(4179, 1)

In [127]:
x_train.head()

,Message
4023,U gd lor go shopping i got stuff to do. U wan ...
1646,I.ll post her out l8r. In class
5422,Ok k..sry i knw 2 siva..tats y i askd..
1254,Mum say we wan to go then go... Then she can s...
3440,"awesome, how do I deal with the gate? Charles ..."


In [128]:
x_test.head()

,Message
3296,Dunno lei ü all decide lor. How abt leona? Oop...
5425,Otherwise had part time job na-tuition..
4685,"My life Means a lot to me, Not because I love ..."
4084,"Good afternoon, my love. It was good to see yo..."
5203,Me fine..absolutly fine


In [129]:
x_test.shape

(1393, 1)

In [130]:
y_train.shape

(4179,)

In [131]:
y_test.shape

(1393,)

In [136]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import FunctionTransformer

# Function to extract the 'Message' column from a DataFrame
def extract_message_column(df):
    # ColumnTransformer will pass a DataFrame (even if it's just one column)
    # so we explicitly select the 'Message' column to get a Series of strings.
    return df['Message']

# Define the preprocessor using ColumnTransformer
# This ensures that only the 'Message' column is sent to the text processing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('text_processing', Pipeline([
            ('selector', FunctionTransformer(extract_message_column, validate=False)),
            ('vectorizer', CountVectorizer())]
                                     ), ['Message']) # Apply this pipeline specifically to the 'Message' column
    ],
    remainder='passthrough' # If there were other columns, they would be passed through or dropped
)

# Create the full pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor), # Our custom preprocessor step
    ('model', MultinomialNB())      # The Multinomial Naive Bayes model
])

# Fit the pipeline
pipe.fit(x_train, y_train)

# Make predictions
y_pred=pipe.predict(x_test)
accuracy_score(y_test,y_pred)

0.9856424982053122